In [1]:
"""
Trabalho computacional sobre algoritmos evolutivos para solução de sistemas de equações não lineares 
"""

'\nTrabalho computacional sobre algoritmos evolutivos para solução de sistemas de equações não lineares \n'

In [2]:
"""
Sistema de equações não lineares
"""
import numpy as np
import random


class EquationSystem:
    def __init__(self, equations):
        self.equations = equations

    def evaluate(self, x):
        return [np.abs(equation(x)) for equation in self.equations]


p1 = EquationSystem([
    lambda x: 0.8*(x[0]**2 + x[0] -1)*x[2] + 0.12*x[0]**2 + 2.16*x[0] - 0.12,
    lambda x: (1 + x[0]**2)*x[3] + 0.4*x[0]**2 - 1.6*x[0] - 0.4,
    lambda x: (1 + x[0]**2)*x[4] + x[0]**2 - 1,
    lambda x: (1 + x[0]**2)*x[5] + 0.8*(x[0]**2 + x[0] -1),
    lambda x: x[4]*x[6] - 0.02*x[5] - x[4] - x[2]*x[3] - 0.16*x[0],
    lambda x: x[6]**2 - 2*x[3]*x[6] + x[5]**2 + x[3]**2 - x[1]**2,
    lambda x: x[7] - x[1]*x[2],
    lambda x: 0.0476*x[2]*x[7]**12 + x[2] - 2.104
])

# p1 bounds
p1_lb = [-3, -1, -2, -1, -1, -0.5, -1.5, -1.5]
p1_ub = [1, 1, 2, 1, 1, 0.5, 1.5, 1.5]

In [10]:
# Real-Coded Genetic Algorithm

def generate_individual(upper_bound: list, lower_bound: list) -> np.array:
    return np.array(
        [
            random.uniform(lower_bound[i], upper_bound[i])
            for i in range(len(upper_bound))
        ]
    )


def generate_population(upper_bound: list, lower_bound: list, pop_size) -> list:
    return [generate_individual(upper_bound, lower_bound) for _ in range(pop_size)]


# evaluate Using the penalty method
def evaluate_penalty(individual, equation_system):
    result = equation_system.evaluate(individual)
    penalty_sum = 0
    for r in result:
        penalty_sum += max(0, r) ** 2
    return penalty_sum

"""
 Um exemplo específico para o caso de codificação em ponto
flutuante é o chamado crossover aritmético (MICHALEWICZ, 1996). Este operador é
definido como uma fusão de dois vetores (cromossomos): se x1 e x2 são dois indivíduos
selecionados para crossover, os dois filhos resultantes serão x_1^line = alpha*x_1 + (1 - alpha)*x_2 e
x_2^line = (1 - alpha)*x_1 + alpha*x_2, sendo alpha um número aleatório pertencente ao intervalo [0, 1]. 
"""

def arithmetical_crossover(parent1, parent2, alpha):
    parent1, _ = parent1
    parent2, _ = parent2
    offspring1 = alpha * parent1 + (1 - alpha) * parent2
    offspring2 = (1 - alpha) * parent1 + alpha * parent2
    return (offspring1, None), (offspring2, None)

"""
o caso de problemas com codificação em ponto flutuante, o operador de mutação
mais popular é a mutação gaussiana (MICHALEWICZ & S CHOENAUER , 1996), modificando
todos os componentes de um cromossomo x = [x1 … xn] na forma:
x^line = x + N(0, sigma)

Sendo N(0, sigma) um vetor de variáveis aleatórias independentes, com distribuição normal,
média zero e desvio padrão sigma
"""

def gaussian_mutation(individual, sigma):
    ind, fitness = individual
    return ind + np.random.normal(0, sigma, len(ind)), fitness


def roulette_wheel_selection(evaluated_population):
    total_fitness = 0
    for individual, fitness in evaluated_population:
        total_fitness += 1/fitness
    r = random.uniform(0, total_fitness)
    acc = 0
    for individual, fitness in evaluated_population:
        acc += 1/fitness
        if acc >= r:
            return individual, fitness
    return evaluated_population[-1]


def real_coded_genetic_algorithm(equation_system, lower_bound, upper_bound, pop_size, max_gen, pc, pm, alpha, sigma, elitisim=True):
    """
    Real-Coded Genetic Algorithm
    :param equation_system: EquationSystem
    :param lower_bound: list
    :param upper_bound: list
    :param pop_size: int
    :param max_gen: int
    :param pc: float - Crossover probability
    :param pm: float - Mutation probability
    :param alpha: float - Crossover parameter
    :param sigma: float - Mutation parameter
    """
    # pop_size must be even
    if pop_size % 2 != 0:
        raise ValueError("pop_size must be even")

    population = generate_population(upper_bound, lower_bound, pop_size)
    population = [(individual, evaluate_penalty(individual, equation_system)) for individual in population]

    history = []
    for gen in range(max_gen):
        
        print(f"Generation {gen+1}", end="\r")
        # Evaluate
        evaluated_population = []
        # print(population)
        for individual, _ in population:
            fitness = evaluate_penalty(individual, equation_system)
            evaluated_population.append((individual, fitness))
        evaluated_population = sorted(evaluated_population, key=lambda x: x[1])

        # Keep 1 elite individual
        if elitisim:
            elite = evaluated_population[0]
        else:
            elite = None
        
        # Selection
        selected_population = []
        for _ in range(pop_size):
            selected_population.append(roulette_wheel_selection(evaluated_population))
        # Crossover
        offspring_population = []
        for i in range(0, pop_size, 2):
            if random.random() < pc:
                # print(selected_population[i], selected_population[i+1])
                offspring1, offspring2 = arithmetical_crossover(selected_population[i], selected_population[i+1], alpha)
                offspring_population.append(offspring1)
                offspring_population.append(offspring2)
            else:
                offspring_population.append(selected_population[i])
                offspring_population.append(selected_population[i+1])

        # Mutation
        mutated_population = []
        for individual in offspring_population:
            if random.random() < pm:
                mutated_population.append(gaussian_mutation(individual, sigma))
            else:
                mutated_population.append(individual)

        # Replace population
        population = mutated_population
        if elite:
            population[0] = elite
        
        best_fit = evaluated_population[0][1]
        history.append(best_fit)

    return evaluated_population[0], history


In [16]:
result, history = real_coded_genetic_algorithm(p1, p1_lb, p1_ub, 100, 2000, 0.8, 0.1, 0.5, 0.1, elitisim=True)
best_individual, best_fitness = result

In [17]:
result

(array([0.29296434, 1.08628568, 1.10734142, 0.64671175, 0.87838582,
        0.39807136, 1.69366416, 1.27484892]),
 0.06148337322982514)

In [18]:
# não tá tudo dando 0 ainda, tem q melhorar o algoritmo
p1.evaluate(best_individual)

[0.02720874130593276,
 0.13219391072838693,
 0.0396041085798986,
 0.06472897853038251,
 0.16166166340251234,
 0.07455358982413274,
 0.07195979324954038,
 0.025269715084727196]

In [19]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(y=history, mode='lines'))
fig.show()